<a href="https://colab.research.google.com/github/jasmeen2105/Pneumonia_Detection/blob/main/Pneumonia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle

In [3]:
!mv kaggle.json ~/.kaggle/

mv: cannot stat 'kaggle.json': No such file or directory


In [4]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
!kaggle datasets download -d [/content/kaggle (1).json]

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `kaggle datasets download -d [/content/kaggle (1).json]'


In [6]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 99% 2.28G/2.29G [00:14<00:00, 253MB/s]
100% 2.29G/2.29G [00:14<00:00, 165MB/s]


In [7]:
import zipfile

with zipfile.ZipFile("chest-xray-pneumonia.zip", "r") as zip_ref:
    zip_ref.extractall("chest_xray")

In [8]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [15]:
# Define dataset path
data_dir = '/content/chest_xray/chest_xray'
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
val_dir = os.path.join(data_dir, "val")

In [16]:
# Image size & batch size
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

In [17]:
# Data Augmentation & Preprocessing
datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, horizontal_flip=True)

train_generator = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
test_generator = datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
val_generator = datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [18]:
# CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Train the model
epochs = 10
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 332s 2s/step - accuracy: 0.7596 - loss: 0.5492 - val_accuracy: 0.7500 - val_loss: 0.9231
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 328s 2s/step - accuracy: 0.9004 - loss: 0.2491 - val_accuracy: 0.6875 - val_loss: 1.5768
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.9048 - loss: 0.2217 - val_accuracy: 0.6875 - val_loss: 0.8772
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 330s 2s/step - accuracy: 0.9097 - loss: 0.2211 - val_accuracy: 0.6250 - val_loss: 1.1178
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 330s 2s/step - accuracy: 0.9318 - loss: 0.1778 - val_accuracy: 0.6250 - val_loss: 1.0262
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 381s 2s/step - accuracy: 0.9272 - loss: 0.1726 - val_accuracy: 0.6250 - val_loss: 1.1578
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 381s 2s/step - accuracy: 0.9289 - loss: 0.1675 - val_accuracy: 0.6250 - val_loss: 1.1288
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 329s 2s/step - accuracy: 0.9394 - loss: 0.1663 - val_accu

In [21]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 762ms/step - accuracy: 0.8004 - loss: 0.6944
Test Accuracy: 81.25%


In [22]:
# Function to predict pneumonia
def predict_pneumonia(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Reshape for model
    prediction = model.predict(img)[0][0]
    return "Pneumonia Detected" if prediction > 0.5 else "Normal"

In [25]:
# Test prediction
sample_image = "/content/chest_xray/chest_xray/test/NORMAL/IM-0006-0001.jpeg"
print(predict_pneumonia(sample_image))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Normal
